In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import gmean
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
import matplotlib.pyplot as plt
import seaborn as sns

#%% Code so the same scripts can be used as notebooks and as part of a pipeline
try:
    if __IPYTHON__:
        import sys
        sys.path.insert(0,'./bin')
except:
    pass

from pipeline_utils.ic_utils import get_IC, get_observed_variance, get_pb_variance
from pipeline_utils.vrs_utils import get_vrs, get_cvrs

In [ ]:
dataset = "somatosensory_rpkm_suppl"
prefix = "Pcdh"

In [ ]:
data_df = pd.read_csv(dataset+".csv", index_col="gene_id")

In [ ]:
thresholds_df = {x: pd.read_csv("{}/intermediate/{}/optimal_thresholds.csv".format(dataset,x), index_col="gene_id") for x in ["vrs", "3max", "geomean"]}

In [ ]:
if dataset != "olfactory_dataset":
    thresholds_df["fitted"] = pd.read_csv("{}/mathematica/fitted_distributions_thresholds.csv".format(dataset), index_col="gene_id")

In [ ]:
def get_np_func(name):
    if (name == "min"):
        return np.nanmin
    elif (name == "max"):
        return np.nanmax
    elif (name == "median"):
        return np.nanmedian

def get_threshold(genes, take_n=3, cutoff=0.0, bottom_func="min", top_func="max"):
    bottom = [row[row > cutoff].sort_values()[:take_n].mean() for _, row in genes.iterrows()]
    top = [row[row > cutoff].sort_values()[-take_n:].mean() for _, row in genes.iterrows()]
    return gmean([get_np_func(bottom_func)(bottom), get_np_func(top_func)(top)])

def get_max_min_ratios(genes, take_n=3, cutoff=0.0):
    bottom = np.array([row[row > cutoff].sort_values()[:take_n].mean() for _, row in genes.iterrows()])
    top = np.array([row[row > cutoff].sort_values()[-take_n:].mean() for _, row in genes.iterrows()])
    return pd.Series(top / bottom, index=genes.index)

def dichotomise(df, threshold):
    if(isinstance(threshold, pd.Series)):
        for gene_id, row in df.iterrows():
            row = dichotomise(row, threshold.loc[gene_id])
    else:
        df[df < threshold] = 0
        df[df >= threshold] = 1
    return df

In [ ]:
def pipeline(kwargs, genes_df, threshold=None):
    if threshold is None:
        threshold = get_threshold(genes_df, **kwargs)
    if(not isinstance(threshold, pd.Series)):
        kwargs["uncorrected_threshold"] = threshold
        threshold = max(0.5,threshold)
        kwargs["threshold"] = threshold
    dichotomised_df = dichotomise(genes_df.copy(), threshold)
    kwargs["obs_var"] = obs_var = get_observed_variance(dichotomised_df)
    kwargs["pb_var"] = pb_var = get_pb_variance(dichotomised_df)
    kwargs["ic"] = ic = get_IC(obs_var, pb_var)
    kwargs["expression_mean"] = expression_mean = dichotomised_df.sum(axis=0).mean()
    return kwargs, dichotomised_df

def heatmap_pipline(kwargs, genes_df):
    output, dichotomised_df = pipeline(kwargs, genes_df)
    plt.figure(figsize=(16,10))
    sns.heatmap(
        dichotomised_df.loc[dichotomised_df.sum(axis=1) > 0],
        linewidths=.5,linecolor="grey"
    )
    display(output)

In [ ]:
if(prefix == "Pcdha" or prefix == "Pcdh"):
    genes_df = data_df.loc[data_df.index.str.startswith(prefix)].drop_duplicates().drop("Pcdha4-g")
if(prefix == "clustered_pcdh"):
    genes_df = data_df.loc[data_df.index.str.startswith("Pcdha") | data_df.index.str.startswith("Pcdhb") | data_df.index.str.startswith("Pcdhg")].drop_duplicates().drop("Pcdha4-g")
else:
    genes_df = data_df.loc[data_df.index.str.startswith(prefix)].drop_duplicates()
    
genes_df

In [ ]:
for x in ["vrs", "3max", "geomean", "fitted"] if dataset != "olfactory_dataset" else ["vrs", "3max", "geomean"]:
    print(x)
    selected_thresholds = thresholds_df[x].loc[genes_df.index]
    x = np.arange(np.nanmin(selected_thresholds), np.nanmax(selected_thresholds), 0.5)
    cvrs = [get_cvrs(selected_thresholds[~selected_thresholds.uncorrected_threshold.isna()].uncorrected_threshold, _x) * 2 for _x in x]
    vrs = [get_vrs(selected_thresholds[~selected_thresholds.uncorrected_threshold.isna()].uncorrected_threshold, _x) * 2 for _x in x]

    # np.log(selected_thresholds[~selected_thresholds.uncorrected_threshold.isna()].uncorrected_threshold).hist(bins=50)
    # plt.show()
    (selected_thresholds[~selected_thresholds.uncorrected_threshold.isna()].uncorrected_threshold).hist(bins=50)
    plt.plot(x, vrs, label="VRS")
    print("VRS: ",x[np.argmin(vrs)])
    # plt.plot(x, cvrs, label="CVRS")
    # print("CVRS: ",x[np.nanargmin(cvrs)])
    plt.show()

In [ ]:
output = {
    "minmax_00": pipeline({
        "take_n": 3,
        "cutoff": 0,
        "bottom_func": "min",
        "top_func": "max"
    }, genes_df)[0],
    "minmax_05": pipeline({
        "take_n": 3,
        "cutoff": 0.4999,
        "bottom_func": "min",
        "top_func": "max"
    }, genes_df)[0],
    "median_00": pipeline({
        "take_n": 3,
        "cutoff": 0,
        "bottom_func": "median",
        "top_func": "median"
    }, genes_df)[0],
    "median_05": pipeline({
        "take_n": 3,
        "cutoff": 0.4999,
        "bottom_func": "median",
        "top_func": "median"
    }, genes_df)[0],
    "mean_vrs": pipeline(
        {},
        genes_df,
        threshold=thresholds_df["vrs"].loc[genes_df.index].uncorrected_threshold.mean()
    )[0],
    "median_vrs": pipeline(
        {},
        genes_df,
        threshold=thresholds_df["vrs"].loc[genes_df.index].uncorrected_threshold.median()
    )[0],
    "mean_3max": pipeline(
        {},
        genes_df,
        threshold=thresholds_df["3max"].loc[genes_df.index].uncorrected_threshold.mean()
    )[0],
    "median_3max": pipeline(
        {},
        genes_df,
        threshold=thresholds_df["3max"].loc[genes_df.index].uncorrected_threshold.median()
    )[0],
    "mean_geomean": pipeline(
        {},
        genes_df,
        threshold=thresholds_df["geomean"].loc[genes_df.index].uncorrected_threshold.mean()
    )[0],
    "median_geomean": pipeline(
        {},
        genes_df,
        threshold=thresholds_df["geomean"].loc[genes_df.index].uncorrected_threshold.median()
    )[0]
}

if(dataset != "olfactory_dataset"):
    output["mean_fitted"] = pipeline(
        {},
        genes_df,
        threshold=thresholds_df["fitted"].loc[genes_df.index].uncorrected_threshold.mean()
    )[0]
    output["median_fitted"] = pipeline(
        {},
        genes_df,
        threshold=thresholds_df["fitted"].loc[genes_df.index].uncorrected_threshold.median()
    )[0]

uniform_output_df = pd.DataFrame(output).iloc[4:].T

In [ ]:
vals = genes_df[genes_df >= 0.5].values.flatten()
vals = sorted(vals[~np.isnan(vals)])

In [ ]:
ics = pd.DataFrame([pipeline({}, genes_df, threshold=x)[0] for x in tqdm_notebook(vals)])

In [ ]:
sns.lineplot(vals, ics.pb_var, label="PB")
sns.lineplot(vals, ics.obs_var, label="Observed")
plt.yscale("log")
plt.xscale("log")
plt.show()

# sns.lineplot(vals, ics.obs_var-ics.pb_var, label="Difference")
# sns.lineplot(vals, np.zeros(len(vals)), label="Difference")
# plt.xscale("log")
# plt.show()

sns.lineplot(vals, ics.ic)
sns.lineplot(vals, np.ones(len(vals)))
# plt.yscale("log")
plt.xscale("log")
plt.show()

sns.lineplot(vals, ics.expression_mean)
sns.lineplot(vals, np.ones(len(vals)))
# plt.yscale("log")
plt.xscale("log")
plt.show()



In [ ]:
ics

In [ ]:
uniform_df = dichotomise(genes_df.copy(), max(0.5, get_threshold(genes_df)))

In [ ]:
isinstance(thresholds_df["vrs"].loc[genes_df.index].threshold, pd.Series)

In [ ]:
separate = {x: pipeline({}, genes_df.copy(), threshold=thresholds_df[x].threshold)[0] for x in ["vrs", "3max", "geomean"]}
ic_summary = pd.concat([uniform_output_df, pd.DataFrame(separate).T], sort=False)
display(ic_summary)
print(ic_summary.fillna(0).to_latex(float_format="%.2f"))

In [ ]:
if dataset == "olfactory_dataset":
    threshold_summary = pd.DataFrame([thresholds_df[x].loc[genes_df.index].uncorrected_threshold for x in ["vrs", "3max", "geomean"]], index=["vrs", "3max", "geomean"]).T.describe()
else:
    threshold_summary = pd.DataFrame([thresholds_df[x].loc[genes_df.index].uncorrected_threshold for x in ["vrs", "3max", "geomean", "fitted"]], index=["vrs", "3max", "geomean", "fitted"]).T.describe()
display(threshold_summary)
print(threshold_summary.fillna(0).to_latex(float_format="%.2f"))